In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler, FunctionTransformer
import category_encoders as ce

In [2]:
import dagshub
dagshub.init(repo_owner='sourav664', repo_name='real-estate-hybrid-app', mlflow=True)


Accessing as sourav664

Initialized MLflow to track repo "sourav664/real-estate-hybrid-app"

Repository sourav664/real-estate-hybrid-app initialized!

In [3]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow")

In [4]:
print(mlflow.__version__)

2.22.0


In [4]:
# mlflow experiment

mlflow.set_experiment("Exp 3 - LightGBM Hyperparameter Tuning")

<Experiment: artifact_location='mlflow-artifacts:/831f39d69e9440f590c44c0ad5d152b2', creation_time=1764742211921, experiment_id='2', last_update_time=1764742211921, lifecycle_stage='active', name='Exp 3 - LightGBM Hyperparameter Tuning', tags={}>

In [5]:

from sklearn import set_config

set_config(transform_output="pandas")

In [6]:
from pathlib import Path

path = Path.cwd().parent
data_path = path / 'data/raw'

In [7]:
import pandas as pd
df_one = pd.read_csv(f"{data_path}/real_estate.csv")

In [8]:
predictors = ['propertytype', 'bedrooms', 'bathrooms', 'balconies', 'furnished', 'transactiontype', 'ageofcons', 'additionalRooms','region',
              'locality','superbuiltupareasqft','totalfloornumber','price']

In [9]:
df = df_one[predictors]

In [10]:
df_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41860 entries, 0 to 41859
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   propertyid            41860 non-null  int64  
 1   propertytype          41860 non-null  object 
 2   bedrooms              41860 non-null  int64  
 3   bathrooms             41860 non-null  int64  
 4   balconies             41860 non-null  int64  
 5   furnished             41860 non-null  object 
 6   transactiontype       41860 non-null  object 
 7   ageofcons             41860 non-null  object 
 8   additionalRooms       41860 non-null  int64  
 9   region                41860 non-null  object 
 10  locality              41860 non-null  object 
 11  superbuiltupareasqft  41860 non-null  float64
 12  price                 41860 non-null  float64
 13  latitude              41860 non-null  float64
 14  longitude             41860 non-null  float64
 15  url                

In [11]:
df.duplicated().sum(
    
)

np.int64(2139)

In [12]:
df.drop_duplicates(inplace=True)

C:\Users\soura\AppData\Local\Temp\ipykernel_3000\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [15]:
df_one.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41860 entries, 0 to 41859
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   propertyid            41860 non-null  int64  
 1   propertytype          41860 non-null  object 
 2   bedrooms              41860 non-null  int64  
 3   bathrooms             41860 non-null  int64  
 4   balconies             41860 non-null  int64  
 5   furnished             41860 non-null  object 
 6   transactiontype       41860 non-null  object 
 7   ageofcons             41860 non-null  object 
 8   additionalRooms       41860 non-null  int64  
 9   region                41860 non-null  object 
 10  locality              41860 non-null  object 
 11  superbuiltupareasqft  41860 non-null  float64
 12  price                 41860 non-null  float64
 13  latitude              41860 non-null  float64
 14  longitude             41860 non-null  float64
 15  url                

In [16]:
df['propertytype'].value_counts()

propertytype
multistorey apartment      26202
builder floor apartment    10757
residential house           1626
villa                        992
penthouse                    144
Name: count, dtype: int64

In [17]:
X = df.drop(columns=["price"])
y = df["price"]

In [18]:
# train test split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
     

In [19]:
df.shape

(39721, 13)

In [20]:
print("The size of train data is",X_train.shape)
print("The shape of test data is",X_test.shape)

The size of train data is (31776, 12)
The shape of test data is (7945, 12)


In [21]:
ohe_encode = ['transactiontype','region','propertytype','furnished','ageofcons']
target_encode = ['locality']
robust_scaler = ['bedrooms','bathrooms','balconies','superbuiltupareasqft', 'totalfloornumber']

In [22]:
from sklearn.preprocessing import FunctionTransformer

pt = FunctionTransformer(np.log1p, inverse_func=np.expm1, validate=True)


In [23]:
preprocessor = ColumnTransformer(
    [
        ('ohe_encode', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), ohe_encode),
        ('robust_scaler', RobustScaler(), robust_scaler),
    ],remainder="passthrough", n_jobs=-1, force_int_remainder_cols=False,verbose_feature_names_out=False
)


preprocessor.set_output(transform="pandas")

,transformers,"[('ohe_encode', ...), ('robust_scaler', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,-1
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,False
,categories,'auto'
,drop,'first'
,sparse_output,False


In [24]:
from lightgbm import LGBMRegressor
import optuna
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.compose import TransformedTargetRegressor

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [65]:
def objective(trial):
    with mlflow.start_run(nested=True):

        # 🔹 TargetEncoder params
        te_params = {
            "smoothing": trial.suggest_float("te_smoothing", 0.05, 1.0, log=True),
            "min_samples_leaf": trial.suggest_int("te_min_samples_leaf", 10, 200)
        }

        # 🔹 LightGBM params (dataset ~33k)
        lgb_params = {
            "n_estimators": trial.suggest_int("n_estimators", 300, 1200),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
            "max_depth": trial.suggest_int("max_depth", 4, 10),

            "num_leaves": trial.suggest_int("num_leaves", 16, 64),
            # "min_child_samples": trial.suggest_int("min_child_samples", 30, 200),

            "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 5.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 100.0),
            # "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),

            "subsample": trial.suggest_float("subsample", 0.7, 1.0),
            # "feature_fraction": trial.suggest_float("feature_fraction", 0.7, 1.0),
            # "bagging_freq": 1,

            # "max_bin": trial.suggest_int("max_bin", 128, 255),

            "random_state": 42,
            "n_jobs": -1,
        }

        mlflow.log_params({**te_params, **lgb_params})

        # 🔹 Build pipeline
        full_pipe = Pipeline([
            ("target_encoder", ce.TargetEncoder(
                cols=target_encode,
                smoothing=te_params["smoothing"],
                min_samples_leaf=te_params["min_samples_leaf"]
            )),
            ("preprocessor", preprocessor),
            ("model", TransformedTargetRegressor(
                regressor=LGBMRegressor(**lgb_params),
                transformer=pt
            )),
        ])

        # 🔹 CV (leak-safe)
        cv_score = cross_val_score(
            full_pipe,
            X_train,
            y_train,
            cv=5,
            scoring="neg_mean_absolute_error",
            n_jobs=-1
        )

        mean_mae = -cv_score.mean()
        mlflow.log_metric("cv_mae", mean_mae)

        return mean_mae


In [66]:
# create optuna study
study = optuna.create_study(direction="minimize")

with mlflow.start_run(run_name="best_model"):
    # optimize the objective function
    study.optimize(objective,n_trials=100,n_jobs=-1,show_progress_bar=True)

    # log the best parameters
    mlflow.log_params(study.best_params)

    # log the best score
    mlflow.log_metric("best_score",study.best_value)

    # train the model on best parameters
    best_lgbm = LGBMRegressor(**study.best_params)
    model = TransformedTargetRegressor(regressor=best_lgbm,transformer=pt)

    full_pipe = Pipeline([
            ('target_encoder', ce.TargetEncoder(cols=target_encode,  smoothing=0.3,      # IMPORTANT
                                                 min_samples_leaf=20)),
            ('preprocessor', preprocessor),
            ('model', model),
        ])
    

    # train the model
    full_pipe.fit(X_train,y_train)

    # get the predictions
    y_pred_train = full_pipe.predict(X_train)
    y_pred_test = full_pipe.predict(X_test)
    
    
    

    scores = cross_val_score(full_pipe,
                         X_train,
                         y_train,
                         scoring="neg_mean_absolute_error",
                         cv=5,n_jobs=-1)

    # log metrics
    mlflow.log_metric("training_error",mean_absolute_error(y_train,y_pred_train))
    mlflow.log_metric("test_error",mean_absolute_error(y_test,y_pred_test))
    mlflow.log_metric("training_r2",r2_score(y_train,y_pred_train))
    mlflow.log_metric("test_r2",r2_score(y_test,y_pred_test))
    mlflow.log_metric("cross_val",- scores.mean())

    # log the best model
    mlflow.sklearn.log_model(best_lgbm,artifact_path="model")

[I 2025-12-20 11:42:20,819] A new study created in memory with name: no-name-8b5546b6-ba2c-4fc2-a685-d75ca43321a1
  0%|          | 0/100 [00:00<?, ?it/s]

🏃 View run valuable-ant-786 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2/runs/aeea176d01b14a50ae91e50f6cb01ade
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2
🏃 View run treasured-horse-119 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2/runs/6fd78cfb951541cf8abe6f36444ab16d
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2
🏃 View run likeable-sheep-669 at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2/runs/90d18f7622a64fb081da97e381441a56
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2


Best trial: 1. Best value: 0.91259:   1%|          | 1/100 [00:12<21:26, 12.99s/it]

[I 2025-12-20 11:42:34,151] Trial 1 finished with value: 0.9125896830768205 and parameters: {'te_smoothing': 0.40458023175849545, 'te_min_samples_leaf': 148, 'n_estimators': 359, 'learning_rate': 0.08805840764809326, 'max_depth': 6, 'num_leaves': 38, 'min_split_gain': 0.4765029318654146, 'reg_lambda': 41.520803211276885, 'subsample': 0.9195457335884303}. Best is trial 1 with value: 0.9125896830768205.


Best trial: 1. Best value: 0.91259:   2%|▏         | 2/100 [00:15<10:41,  6.55s/it]

...

[I 2025-12-20 11:50:31,193] Trial 99 finished with value: 0.7157472176405049 and parameters: {'te_smoothing': 0.42825333216474093, 'te_min_samples_leaf': 34, 'n_estimators': 693, 'learning_rate': 0.039381810044223954, 'max_depth': 6, 'num_leaves': 62, 'min_split_gain': 0.006615740980427616, 'reg_lambda': 90.49414622090895, 'subsample': 0.7010230111308892}. Best is trial 51 with value: 0.6756337059694066.


c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pan

[LightGBM] [Warning] Unknown parameter: te_smoothing
[LightGBM] [Warning] Unknown parameter: te_min_samples_leaf
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: te_smoothing
[LightGBM] [Warning] Unknown parameter: te_min_samples_leaf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001194 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 596
[LightGBM] [Info] Number of data points in the train set: 31776, number of used features: 23
[LightGBM] [Info] Start training from score 1.335266
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

2025/12/20 11:50:52 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/12/20 11:50:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run best_model at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2/runs/cb6a11af55d94f62ab115ae9092f723d
🧪 View experiment at: https://dagshub.com/sourav664/real-estate-hybrid-app.mlflow/#/experiments/2


In [67]:
study.best_params

{'te_smoothing': 0.4021499908834315,
 'te_min_samples_leaf': 23,
 'n_estimators': 1070,
 'learning_rate': 0.06287046875905022,
 'max_depth': 6,
 'num_leaves': 53,
 'min_split_gain': 0.00790309119294291,
 'reg_lambda': 68.64061504533905,
 'subsample': 0.778718306858155}

In [68]:
study.best_value

0.6756337059694066

In [29]:
lgbm_params = {'n_estimators': 1063,
 'learning_rate': 0.08008165561870688,
 'max_depth': 10,
 'num_leaves': 56,
 'min_child_samples': 45,
 'reg_lambda': 3.8172220680740274,
 'reg_alpha': 0.536819215284686,
 'subsample': 0.8831693731284017}

In [184]:
study.best_params

{'n_estimators': 497,
 'max_depth': 18,
 'learning_rate': 0.3411567207702695,
 'subsample': 0.8211191192401062,
 'min_child_weight': 9,
 'min_split_gain': 0.001980136353254995,
 'reg_lambda': 66.24757875556628}

In [35]:
lgbm_params1 = {'n_estimators': 497,
 'max_depth': 18,
 'learning_rate': 0.3411567207702695,
 'subsample': 0.8211191192401062,
 'min_child_weight': 9,
 'min_split_gain': 0.001980136353254995,
 'reg_lambda': 66.24757875556628}

In [213]:
lgbm_params2 = {'n_estimators': 1054,
 'learning_rate': 0.14855729954723326,
 'max_depth': 6,
 'num_leaves': 33,
 'min_child_samples': 39,
 'reg_lambda': 1.0818200250309242,
 'reg_alpha': 0.9037388279159972,
 'subsample': 0.9899103451330494}

In [58]:
best_lgbm = LGBMRegressor(**lgbm_params1)
model = TransformedTargetRegressor(regressor=best_lgbm,transformer=pt)


full_pipe = Pipeline([
            ('target_encoder', ce.TargetEncoder(cols=target_encode,  smoothing=0.23677073292057985,      # IMPORTANT
    min_samples_leaf=11)),
            ('preprocessor', preprocessor),
            ('model', model),
        ])

In [59]:
full_pipe.fit(X_train,y_train)

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pandas"))
c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\preprocessing\_function_transformer.py:314: UserWarning: When `set_output` is configured to be 'pandas', `func` should return a pandas DataFrame to follow the `set_output` API  or `feature_names_out` should be defined.
  warnings.warn(warn_msg.format("pan

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001245 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 593
[LightGBM] [Info] Number of data points in the train set: 31776, number of used features: 23
[LightGBM] [Info] Start training from score 1.335266
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightG

,steps,"[('target_encoder', ...), ('preprocessor', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,['locality']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,11


In [60]:
y_pred_train = full_pipe.predict(X_train)
y_pred_test = full_pipe.predict(X_test)

In [61]:
print(f"The train error is {mean_absolute_error(y_train,y_pred_train):.2f} cr")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test):.2f} cr")

The train error is 0.48 cr
The test error is 0.59 cr


In [62]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test):.2f}")

The train r2 score is 0.94
The test r2 score is 0.89


In [180]:
df['locality'].nunique()

296

In [23]:
# Creating a pipeline
pipeline = Pipeline([
    ('target_encoder', ce.TargetEncoder(cols=target_encode)),
    ('preprocessor', preprocessor)
    
])

In [24]:
pipeline.fit(X_train,y_train)

c:\Users\soura\OneDrive\Desktop\real-estate-hybrid-app\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:975: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


,steps,"[('target_encoder', ...), ('preprocessor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,verbose,0
,cols,['locality']
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'
,min_samples_leaf,20


In [25]:
X_train_trans = pipeline.transform(X_train)
X_test_trans = pipeline.transform(X_test)

In [26]:
best_lgbm = LGBMRegressor(**study.best_params)
model = TransformedTargetRegressor(regressor=best_lgbm,transformer=pt)

model.fit(X_train_trans,y_train)

NameError: name 'study' is not defined

In [195]:
y_pred_train = model.predict(X_train_trans)
y_pred_test = model.predict(X_test_trans)

In [196]:
print(f"The train error is {mean_absolute_error(y_train,y_pred_train):.2f} cr")
print(f"The test error is {mean_absolute_error(y_test,y_pred_test):.2f} cr")

The train error is 0.50 cr
The test error is 0.60 cr


In [197]:
print(f"The train r2 score is {r2_score(y_train,y_pred_train):.2f}")
print(f"The test r2 score is {r2_score(y_test,y_pred_test):.2f}")

The train r2 score is 0.93
The test r2 score is 0.89
